## 1. 영상 데이터 불러오기

In [1]:
import os
import re

In [2]:
# [필수] 파일 주소 재설정 
MOV_Directory = '../1. Crawling/MOV' 
VideoList = os.listdir(MOV_Directory)

## 2. 영상 파일을 음성 파일로 전환

In [3]:
# !pip install moviepy
import moviepy.editor as mp
from pydub import AudioSegment

In [4]:
# [필수] 파일 주소 재설정 
WAV_Directory = './WAV_streo' 

for i in VideoList:
    i2 = MOV_Directory + '/' + i
    clip = mp.VideoFileClip(i2)
    
    sound_file_name = re.sub(pattern='mp4', repl='wav', string=i)
    sound_file_name = WAV_Directory + '/' + sound_file_name
    clip.audio.write_audiofile(sound_file_name)

chunk:   3%|██                                                            | 86/2550 [00:00<00:02, 859.81it/s, now=None]

MoviePy - Writing audio in ./WAV_streo/readnaversid1355oid056aid0000003463.wav


MoviePy - Done.


chunk:   2%|█▎                                                            | 62/2868 [00:00<00:04, 619.87it/s, now=None]

MoviePy - Writing audio in ./WAV_streo/readnaversid1355oid056aid0000003464.wav


...

MoviePy - Writing audio in ./WAV_streo/readnaversid1355oid056aid0000003468.wav


MoviePy - Done.


chunk:   0%|                                                                        | 0/3178 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in ./WAV_streo/readnaversid1355oid056aid0000003469.wav


KeyboardInterrupt: 

In [5]:
WAV_stereo_List = os.listdir(WAV_Directory)

for i in WAV_stereo_List:
    i2 = WAV_Directory + '/' + i
    sound = AudioSegment.from_wav(i2)
    sound = sound.set_channels(1)
    
    mono_file_name = './WAV_mono/' + i  
    sound.export(mono_file_name, format="wav")